In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import iplot,init_notebook_mode,download_plotlyjs
init_notebook_mode(connected=True)
from datetime import datetime,timedelta

Из запроса получим датафрейм для работы

In [4]:
df=pd.read_csv('query.csv')
df['actual_arrival']=df['actual_arrival'].apply(lambda x : datetime.strptime(x,'%Y-%m-%dT%H:%M:%SZ')) # преобразуем дту в DateTime
df['actual_departure']=df['actual_departure'].apply(lambda x : datetime.strptime(x,'%Y-%m-%dT%H:%M:%SZ'))#преобразуем дту в DateTime
df['flight_time']=(df['actual_arrival']-df['actual_departure']).apply(lambda x :x.total_seconds()/(60*60))# добавим колонку содержащую время полета в часах
df


,flight_id,departure_airport,arrival_airport,model,actual_departure,actual_arrival,departure_city,arrival_city,departure_longitude,departure_latitude,arrival_longitude,arrival_latitude,sum_amount,tickets_sell,flight_time
0,136119,AAQ,SVO,Boeing 737-300,2017-01-03 10:08:00,2017-01-03 11:49:00,Anapa,Moscow,37.347301,45.002102,37.414600,55.972599,1653000.0,113.0,1.683333
1,136120,AAQ,SVO,Boeing 737-300,2017-02-24 10:07:00,2017-02-24 11:46:00,Anapa,Moscow,37.347301,45.002102,37.414600,55.972599,1605400.0,109.0,1.650000
2,136122,AAQ,SVO,Boeing 737-300,2017-01-08 10:08:00,2017-01-08 11:48:00,Anapa,Moscow,37.347301,45.002102,37.414600,55.972599,1431000.0,97.0,1.666667
3,136130,AAQ,SVO,Boeing 737-300,2017-02-13 10:07:00,2017-02-13 11:46:00,Anapa,Moscow,37.347301,45.002102,37.414600,55.972599,1556600.0,107.0,1.650000
4,136131,AAQ,SVO,Boeing 737-300,2017-01-28 10:07:00,2017-01-28 11:46:00,Anapa,Moscow,37.347301,45.002102,37.414600,55.972599,1812800.0,124.0,1.650000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,136936,AAQ,EGO,Sukhoi Superjet-100,2017-01-02 09:28:00,2017-01-02 10:18:00,Anapa,Belgorod,37.347301,45.002102,36.590099,50.643799,632400.0,82.0,0.833333
123,136937,AAQ,EGO,Sukhoi Superjet-100,2017-01-03 09:29:00,2017-01-03 10:19:00,Anapa,Belgorod,37.347301,45.002102,36.590099,50.643799,626100.0,81.0,0.833333
124,136951,AAQ,EGO,Sukhoi Superjet-100,2017-02-02 09:28:00,2017-02-02 10:17:00,Anapa,Belgorod,37.347301,45.002102,36.590099,50.643799,720600.0,90.0,0.816667
125,136953,AAQ,EGO,Sukhoi Superjet-100,2017-02-27 09:27:00,2017-02-27 10:17:00,Anapa,Belgorod,37.347301,45.002102,36.590099,50.643799,765300.0,97.0,0.833333


# Информация по ценам на топливо в аэропортах РФ на 2017 г
Взята с сайта https://favt.gov.ru/dejatelnost-ajeroporty-i-ajerodromy-ceny-na-aviagsm-statistika-aviagsm
и составляет в аэропорту Анапа:
январь 2017 41435 руб/тонна без ндс(48893 с ндс 18%)
февраль 2017 39553 руб/тонна без ндс(46672 с ндс 18%)
декабрь 2017 47101 руб/тонна без ндс(55579 с ндс 18%)
Информация по расходу на топливо взята с википедии и составляет: 
2,4 тыс. кг топлива на 1 час полета для boeing 737-300
1,7 тыс . кг топлива на 1 час полета для sukhoi superjet-100

In [ ]:
дополним датфрейм данными по ценам на топливо и расходу

In [5]:
df['fuel_utilize'] = pd.Series(df.index).apply(lambda x: df['flight_time'].iloc[x]*2.4 if df['model'].iloc[x]
                                               == 'Boeing 737-300' else df['flight_time'].iloc[x]*1.7)  # умножим время полета на расход чтобы получить
# чтобы получить расход за полет для конкретного самолета
# добавим колонку с затратами на  топливо(цена зависит от месяца)
df['fuel_cost'] = pd.Series(df.index).apply(
    lambda x: df['fuel_utilize'].iloc[x]*48893 if df['actual_departure'].iloc[x].month == 1 else None)
df['fuel_cost'] = pd.Series(df.index).apply(lambda x: df['fuel_utilize'].iloc[x]
                                            * 46672 if df['actual_departure'].iloc[x].month == 2 else df['fuel_cost'].iloc[x])
df['fuel_cost'] = pd.Series(df.index).apply(lambda x: df['fuel_utilize'].iloc[x]
                                            * 55579 if df['actual_departure'].iloc[x].month == 12 else df['fuel_cost'].iloc[x])
df['profit']=df['sum_amount']-df['fuel_cost']  # и в конечном итоге добавим столбец разницы в затратах на топливо и собранных денег за полет
df

,flight_id,departure_airport,arrival_airport,model,actual_departure,actual_arrival,departure_city,arrival_city,departure_longitude,departure_latitude,arrival_longitude,arrival_latitude,sum_amount,tickets_sell,flight_time,fuel_utilize,fuel_cost,profit
0,136119,AAQ,SVO,Boeing 737-300,2017-01-03 10:08:00,2017-01-03 11:49:00,Anapa,Moscow,37.347301,45.002102,37.414600,55.972599,1653000.0,113.0,1.683333,4.040000,197527.720000,1.455472e+06
1,136120,AAQ,SVO,Boeing 737-300,2017-02-24 10:07:00,2017-02-24 11:46:00,Anapa,Moscow,37.347301,45.002102,37.414600,55.972599,1605400.0,109.0,1.650000,3.960000,184821.120000,1.420579e+06
2,136122,AAQ,SVO,Boeing 737-300,2017-01-08 10:08:00,2017-01-08 11:48:00,Anapa,Moscow,37.347301,45.002102,37.414600,55.972599,1431000.0,97.0,1.666667,4.000000,195572.000000,1.235428e+06
3,136130,AAQ,SVO,Boeing 737-300,2017-02-13 10:07:00,2017-02-13 11:46:00,Anapa,Moscow,37.347301,45.002102,37.414600,55.972599,1556600.0,107.0,1.650000,3.960000,184821.120000,1.371779e+06
4,136131,AAQ,SVO,Boeing 737-300,2017-01-28 10:07:00,2017-01-28 11:46:00,Anapa,Moscow,37.347301,45.002102,37.414600,55.972599,1812800.0,124.0,1.650000,3.960000,193616.280000,1.619184e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,136936,AAQ,EGO,Sukhoi Superjet-100,2017-01-02 09:28:00,2017-01-02 10:18:00,Anapa,Belgorod,37.347301,45.002102,36.590099,50.643799,632400.0,82.0,0.833333,1.416667,69265.083333,5.631349e+05
123,136937,AAQ,EGO,Sukhoi Superjet-100,2017-01-03 09:29:00,2017-01-03 10:19:00,Anapa,Belgorod,37.347301,45.002102,36.590099,50.643799,626100.0,81.0,0.833333,1.416667,69265.083333,5.568349e+05
124,136951,AAQ,EGO,Sukhoi Superjet-100,2017-02-02 09:28:00,2017-02-02 10:17:00,Anapa,Belgorod,37.347301,45.002102,36.590099,50.643799,720600.0,90.0,0.816667,1.388333,64796.293333,6.558037e+05
125,136953,AAQ,EGO,Sukhoi Superjet-100,2017-02-27 09:27:00,2017-02-27 10:17:00,Anapa,Belgorod,37.347301,45.002102,36.590099,50.643799,765300.0,97.0,0.833333,1.416667,66118.666667,6.991813e+05


In [6]:
'''определим массивы координат широты и долготы аэропортов прибытия'''
t=df.iloc[:,6:12]
lon=list(t['departure_longitude'].unique())
lon2=list(t['arrival_longitude'].unique())
lon.extend(lon2)
lat=list(t['departure_latitude'].unique())
lat2=list(t['arrival_latitude'].unique())
lat.extend(lat2)
lon3=[]
lat3=[]
for i,el in enumerate(lon):
    if i>0:
        lon3.append([lon[0],lon[i]])
for i,el in enumerate(lat):
    if i>0:
        lat3.append([lat[0],lat[i]])
names=['Moscow','Novokuznetsk','Belgorod']    # их всего 3 в датафрейме

визуализируем на карте

In [8]:
fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    lon = lon3[0],
    lat = lat3[0],
    name=names[0],
    marker = {'size': 10}))
for i ,el in enumerate(lon3):
    if i>0:
        fig.add_trace(go.Scattermapbox(
            mode = "markers+lines",
            lon = lon3[i],
            lat = lat3[i],
            name=names[i],
            marker = {'size': 10}))
fig.update_layout(height=1000,
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 57, 'lat': 50},
        'style': "stamen-terrain",
        'center': {'lon': 57, 'lat': 50},
        'zoom': 4})

fig.show()

Визуализируем профит по городам

In [ ]:
fig=make_subplots(rows=1,cols=3)
objects=[go.Box(y=df['profit'][df['arrival_city']==name],name=name) for name in names]
fig.add_traces(objects,
              rows=[1,1,1],
              cols=[1,2,3])

как видим бокс-плот по Новокузнецку полностью пустой.И действительно в базе данных,в таблице tickets_flights отстутсвует информация по выручке и проданным
билетам на данные рейсы в январе-феврале 2017г.Это либо проблема с заполнением базы,либо действительно не продано ни одного билета да Новокузнецка.Поэтому вероятно я бы рекомендовал
отказаться от рейсов в Новокузнецк в зимний период.